In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train = pd.read_csv('/kaggle/input/bike-sharing-demand/train.csv', parse_dates=['datetime'])
test = pd.read_csv('/kaggle/input/bike-sharing-demand/test.csv', parse_dates=['datetime'])
sub = pd.read_csv('/kaggle/input/bike-sharing-demand/sampleSubmission.csv')

In [ ]:
new_tr = train.copy()
new_test = test.copy()

In [ ]:
## 더미변수, 파생변수 생성
new_tr['year'] = new_tr['datetime'].dt.year
new_tr['month'] = new_tr['datetime'].dt.month
new_tr['day'] = new_tr['datetime'].dt.day
new_tr['hour'] = new_tr['datetime'].dt.hour
new_tr['minute'] = new_tr['datetime'].dt.minute
new_tr['second'] = new_tr['datetime'].dt.second
new_tr['dayofweek'] = new_tr['datetime'].dt.dayofweek

In [ ]:
new_test['year'] = new_test['datetime'].dt.year
new_test['month'] = new_test['datetime'].dt.month
new_test['day'] = new_test['datetime'].dt.day
new_test['hour'] = new_test['datetime'].dt.hour
new_test['minute'] = new_test['datetime'].dt.minute
new_test['second'] = new_test['datetime'].dt.second
new_test['dayofweek'] = new_test['datetime'].dt.dayofweek

In [ ]:
col_names = ['year','month','day','hour','dayofweek']
i = 0
plt.figure(figsize=(12,35))  ##전체 그래프 크기 지정

for name in col_names: ## 컬럼명으로 반복
  i = i+1
  plt.subplot(6,2,i)  ##2행2열, i = 1,2,3,4 (왼쪽 상단부터 시계방향으로 순번 지정)
  sns.countplot(x = name, data = new_tr)
  plt.title("train feature")
    
  i = i+1
  plt.subplot(6,2,i)  ##2행2열, i = 1,2,3,4 (왼쪽 상단부터 시계방향으로 순번 지정)
  sns.countplot(x = name, data = new_test)
  plt.title("test feature")
plt.show()

In [ ]:
plt.figure(figsize=(15,10))
g = sns.heatmap(new_tr.corr(), annot=True, fmt=".2f", cmap="coolwarm", cbar=False)

In [ ]:
feature_names = [ 'season', 'holiday', 'workingday', 'weather', 
                  'temp', 'atemp', 'humidity', 'windspeed', 
                  "year", "hour", "dayofweek"]  # 공통 변수

In [ ]:
train

In [ ]:
from lightgbm import LGBMRegressor # GBT
from sklearn.model_selection import train_test_split

In [ ]:
new_tr['log_count'] = np.log1p(new_tr['count'])

In [ ]:
X = new_tr[feature_names]
y = new_tr['log_count']

X_test_last = new_test[feature_names]

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
model_lgbm = LGBMRegressor()

In [ ]:
model_lgbm.fit(X_train, y_train)

In [ ]:
model_lgbm.score(X_test, y_test)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model_random = RandomForestRegressor()

In [ ]:
model_random.fit(X_train, y_train)

In [ ]:
model_random.score(X_test, y_test)

In [ ]:
from sklearn.model_selection import cross_val_score # 교차검증
scores = cross_val_score(model_random, X_test, y_test)
print("{} 교차 검증 점수 : {}".format(model_random,scores))

In [ ]:
from sklearn.model_selection import LeaveOneOut, cross_val_score, KFold

In [ ]:
loo = LeaveOneOut()

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import lightgbm as lgb
import time

In [ ]:
def model_val(model_name, model_obj):
    now_time = time.time()  # 시작 시간 
    
    model_obj.fit(X_train, y_train)

    scores = cross_val_score(model_obj, X_test, y_test, cv=5, 
                         scoring='neg_mean_squared_error')
    avg_score = np.abs( scores.mean() )
    print("MSE 평균 : ", avg_score)
    
    pro_time = time.time() - now_time
    print("수행 시간 : {0:.3f}".format(pro_time))
    print("{} Score : {}".format(model_name, avg_score))
    return avg_score, pro_time   

In [ ]:
model_list = ["RandomForestRegressor",  "xgb_basic", 
              "lightgbm-model1", "lightgbm-model2"]

exe_model = []
model_score = []
model_time = []

In [ ]:
m_name = model_list[0]

if m_name not in exe_model:
    model = RandomForestRegressor(random_state=30)
    mse_score, p_time = model_val(m_name, model)
    
    exe_model.append(m_name) # 실행 완료
    model_score.append(mse_score)
    model_time.append(p_time)
else:
    print(f"{m_name} 이미 실행 완료")

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor

In [ ]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', 
                colsample_bytree = 0.3, # 각나무마다 사용하는 feature 비율
                learning_rate = 0.1,
                max_depth = 3, 
                alpha = 0.1, 
                n_estimators = 1000)
mse_score, p_time = model_val('xgb_basic', xg_reg)
exe_model.append('xgb_basic') # 실행 완료
model_score.append(mse_score)
model_time.append(p_time)

In [ ]:
m_lgbm1 = lgb.LGBMRegressor()
mse_score, p_time = model_val('lightgbm-model1', m_lgbm1)
exe_model.append('lightgbm-model1') # 실행 완료
model_score.append(mse_score)
model_time.append(p_time)

In [ ]:
hyperparameters = {'boosting_type': 'gbdt', 
                   'colsample_bytree': 0.7250136792694301, 
                   'is_unbalance': False, 
                   'learning_rate': 0.013227664889528229,
                   'min_child_samples': 20, 
                   'num_leaves': 56, 
                   'reg_alpha': 0.7543896477745794, 
                   'reg_lambda': 0.07152751159655985, 
                   'subsample_for_bin': 240000, 
                   'subsample': 0.5233384321711397, 
                   'n_estimators': 2000}

m_lgbm2 = lgb.LGBMRegressor(**hyperparameters)
mse_score, p_time = model_val('lightgbm-model2', m_lgbm2)
exe_model.append('lightgbm-model2') # 실행 완료
model_score.append(mse_score)
model_time.append(p_time)

In [ ]:
print(exe_model)
print(model_score)
print(model_time)

In [ ]:
exe_model = ['rf', 'xgb', 'lgb_1', 'lgb_2']

In [ ]:
sns.barplot(x=exe_model, y=model_score)


In [ ]:
sns.barplot(x=exe_model, y=model_time)

In [ ]:
hyperparameters = {'boosting_type': 'gbdt', 
                   'colsample_bytree': 0.7250136792694301, 
                   'is_unbalance': False, 
                   'learning_rate': 0.013227664889528229,
                   'min_child_samples': 20, 
                   'num_leaves': 56, 
                   'reg_alpha': 0.7543896477745794, 
                   'reg_lambda': 0.07152751159655985, 
                   'subsample_for_bin': 240000, 
                   'subsample': 0.5233384321711397, 
                   'n_estimators': 2000}

# 최종 모델 선택 및 제출
m_lgbm2 = lgb.LGBMRegressor(**hyperparameters)
m_lgbm2.fit(X_train, y_train)

pred = m_lgbm2.predict(X_test_last)
sub['count'] = np.expm1(pred)
sub.to_csv("submission.csv", index=False)